# LSTM Deep Learning Model for Turbidity Prediction

## AAI-530 Final Project - Machine Learning Method 1

This notebook implements an LSTM (Long Short-Term Memory) neural network built from scratch using TensorFlow/Keras to predict future water turbidity levels based on historical sensor data.

**Objective**: Time series forecasting - Predict turbidity 24 hours ahead

**Why LSTM?**
- LSTMs excel at capturing long-term dependencies in sequential data
- Water quality parameters exhibit temporal patterns (daily, seasonal)
- Can learn complex non-linear relationships between multiple sensor inputs

**Model Architecture**:
- Built from scratch using TensorFlow/Keras
- Multi-layer LSTM with dropout for regularization
- Input: Sequences of historical sensor readings
- Output: Predicted turbidity value


In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os

# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print("Libraries loaded successfully!")


ModuleNotFoundError: No module named 'pandas'

## 1. Load and Prepare Data


In [ ]:
# Load raw data and preprocess (if processed data not available)
DATA_DIR = '../archive'

# Load all station data
def load_all_stations(data_dir):
    """Load and combine data from all monitoring stations."""
    all_data = []
    for filename in os.listdir(data_dir):
        if filename.endswith('.csv'):
            filepath = os.path.join(data_dir, filename)
            station_name = filename.replace('_joined.csv', '').replace('_', ' ').title()
            df = pd.read_csv(filepath)
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
            df['Station'] = station_name
            all_data.append(df)
    return pd.concat(all_data, ignore_index=True)

# Load data
df = load_all_stations(DATA_DIR)
print(f"Loaded {len(df):,} records from {df['Station'].nunique()} stations")
print(f"Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")


In [ ]:
# Select station with most complete Turbidity data for LSTM training
station_turbidity_counts = df.groupby('Station')['Turbidity'].apply(lambda x: x.notna().sum())
best_station = station_turbidity_counts.idxmax()
print(f"Selected station for LSTM: {best_station}")
print(f"Available Turbidity readings: {station_turbidity_counts[best_station]:,}")

# Filter to selected station
df_station = df[df['Station'] == best_station].copy()
df_station = df_station.sort_values('Timestamp').reset_index(drop=True)
print(f"\nStation data shape: {df_station.shape}")
df_station.head()


In [ ]:
# Data preprocessing for LSTM
# Select features for prediction
feature_cols = ['Turbidity', 'Conductivity', 'Temp']
available_features = [col for col in feature_cols if col in df_station.columns]
print(f"Available features: {available_features}")

# Filter to rows with Turbidity data (our target variable)
df_lstm = df_station[['Timestamp'] + available_features].copy()
df_lstm = df_lstm.dropna(subset=['Turbidity'])

# Interpolate missing values in other features
for col in available_features:
    df_lstm[col] = df_lstm[col].interpolate(method='linear', limit=6)

# Drop any remaining NaN rows
df_lstm = df_lstm.dropna()
df_lstm = df_lstm.reset_index(drop=True)

print(f"\nLSTM dataset shape: {df_lstm.shape}")
print(f"Date range: {df_lstm['Timestamp'].min()} to {df_lstm['Timestamp'].max()}")
df_lstm.describe()


## 2. Feature Scaling and Sequence Creation


In [ ]:
# Scale features using MinMaxScaler
# This is important for LSTM as it helps with gradient computation

scaler = MinMaxScaler(feature_range=(0, 1))

# Get feature values (exclude timestamp)
feature_data = df_lstm[available_features].values
scaled_data = scaler.fit_transform(feature_data)

print(f"Scaled data shape: {scaled_data.shape}")
print(f"Feature range after scaling: [{scaled_data.min():.4f}, {scaled_data.max():.4f}]")

# Get the index of Turbidity (our target variable)
target_idx = available_features.index('Turbidity')
print(f"Target variable (Turbidity) index: {target_idx}")


In [ ]:
def create_sequences(data, target_col_idx, sequence_length=24, forecast_horizon=24):
    """
    Create sequences for LSTM model training.
    
    Parameters:
    -----------
    data : np.array
        Scaled feature data
    target_col_idx : int
        Index of target column in data
    sequence_length : int
        Number of past time steps to use as input (lookback window)
    forecast_horizon : int
        Number of steps ahead to predict
        
    Returns:
    --------
    X : np.array
        Input sequences of shape (samples, sequence_length, features)
    y : np.array
        Target values of shape (samples,)
    """
    X, y = [], []
    
    for i in range(sequence_length, len(data) - forecast_horizon + 1):
        # Input: sequence of past values
        X.append(data[i - sequence_length:i])
        # Output: target value at forecast_horizon steps ahead
        y.append(data[i + forecast_horizon - 1, target_col_idx])
    
    return np.array(X), np.array(y)

# Hyperparameters
SEQUENCE_LENGTH = 48  # Use 48 hours of historical data
FORECAST_HORIZON = 24  # Predict 24 hours ahead

# Create sequences
X, y = create_sequences(scaled_data, target_idx, SEQUENCE_LENGTH, FORECAST_HORIZON)

print(f"Input sequences shape (X): {X.shape}")
print(f"  - Samples: {X.shape[0]}")
print(f"  - Sequence length (time steps): {X.shape[1]}")
print(f"  - Features: {X.shape[2]}")
print(f"\nTarget shape (y): {y.shape}")


In [ ]:
# Split data into training, validation, and test sets
# For time series, we use sequential split (not random) to maintain temporal order

train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)

X_train = X[:train_size]
y_train = y[:train_size]

X_val = X[train_size:train_size + val_size]
y_val = y[train_size:train_size + val_size]

X_test = X[train_size + val_size:]
y_test = y[train_size + val_size:]

print("Data Split Summary:")
print(f"  Training set:   {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Validation set: {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test set:       {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")


## 3. Build LSTM Model from Scratch

Building a multi-layer LSTM neural network using TensorFlow/Keras. The architecture includes:
- Input layer matching our sequence shape
- Two LSTM layers with dropout for regularization
- Dense output layer for prediction


In [ ]:
def build_lstm_model(sequence_length, n_features, units_1=64, units_2=32, dropout_rate=0.2):
    """
    Build an LSTM model from scratch for time series prediction.
    
    Architecture:
    - Input Layer: (sequence_length, n_features)
    - LSTM Layer 1: units_1 neurons with return_sequences=True
    - Dropout Layer 1: dropout_rate
    - LSTM Layer 2: units_2 neurons
    - Dropout Layer 2: dropout_rate
    - Dense Output Layer: 1 neuron (prediction)
    
    Parameters:
    -----------
    sequence_length : int
        Number of time steps in input sequence
    n_features : int
        Number of features per time step
    units_1 : int
        Number of LSTM units in first layer
    units_2 : int
        Number of LSTM units in second layer
    dropout_rate : float
        Dropout rate for regularization
        
    Returns:
    --------
    keras.Model
        Compiled LSTM model
    """
    model = Sequential([
        # Input layer - explicitly define input shape
        Input(shape=(sequence_length, n_features)),
        
        # First LSTM layer - returns sequences for stacking
        LSTM(units=units_1, 
             return_sequences=True,
             activation='tanh',
             recurrent_activation='sigmoid',
             name='lstm_layer_1'),
        Dropout(dropout_rate, name='dropout_1'),
        
        # Second LSTM layer - returns final output only
        LSTM(units=units_2,
             return_sequences=False,
             activation='tanh',
             recurrent_activation='sigmoid',
             name='lstm_layer_2'),
        Dropout(dropout_rate, name='dropout_2'),
        
        # Dense hidden layer
        Dense(units=16, activation='relu', name='dense_hidden'),
        
        # Output layer - single value prediction
        Dense(units=1, activation='linear', name='output')
    ])
    
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',  # Mean Squared Error for regression
        metrics=['mae']  # Mean Absolute Error
    )
    
    return model

# Build the model
n_features = X_train.shape[2]
model = build_lstm_model(SEQUENCE_LENGTH, n_features, units_1=64, units_2=32, dropout_rate=0.2)

# Display model architecture
print("LSTM Model Architecture:")
print("=" * 60)
model.summary()


## 4. Train the Model


In [ ]:
# Define callbacks for training
callbacks = [
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    # Reduce learning rate when validation loss plateaus
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.0001,
        verbose=1
    ),
    # Save best model
    ModelCheckpoint(
        filepath='../models/lstm_turbidity_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=0
    )
]

# Training hyperparameters
EPOCHS = 100
BATCH_SIZE = 32

print("Starting model training...")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print("=" * 60)


In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\nTraining completed!")
print(f"Final training loss: {history.history['loss'][-1]:.6f}")
print(f"Final validation loss: {history.history['val_loss'][-1]:.6f}")


In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].set_title('Model Loss During Training', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# MAE plot
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('MAE', fontsize=12)
axes[1].set_title('Mean Absolute Error During Training', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/lstm_training_history.png', dpi=150, bbox_inches='tight')
plt.show()


## 5. Model Evaluation


In [ ]:
# Make predictions on test set
y_pred_scaled = model.predict(X_test, verbose=0)

# Inverse transform predictions to original scale
# We need to create a dummy array with the same shape as original features
def inverse_transform_target(scaled_pred, scaler, target_idx, n_features):
    """Convert scaled predictions back to original scale."""
    # Create dummy array
    dummy = np.zeros((len(scaled_pred), n_features))
    dummy[:, target_idx] = scaled_pred.flatten()
    
    # Inverse transform
    original = scaler.inverse_transform(dummy)
    return original[:, target_idx]

y_pred = inverse_transform_target(y_pred_scaled, scaler, target_idx, len(available_features))
y_test_original = inverse_transform_target(y_test.reshape(-1, 1), scaler, target_idx, len(available_features))

# Calculate metrics
mse = mean_squared_error(y_test_original, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_original, y_pred)
r2 = r2_score(y_test_original, y_pred)

print("Model Evaluation Metrics (Test Set):")
print("=" * 50)
print(f"  Mean Squared Error (MSE):  {mse:.4f}")
print(f"  Root MSE (RMSE):           {rmse:.4f} NTU")
print(f"  Mean Absolute Error (MAE): {mae:.4f} NTU")
print(f"  R² Score:                  {r2:.4f}")
print("=" * 50)


In [ ]:
# Plot predictions vs actual values
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Scatter plot: Predicted vs Actual
axes[0, 0].scatter(y_test_original, y_pred, alpha=0.5, s=20, c='steelblue')
axes[0, 0].plot([y_test_original.min(), y_test_original.max()], 
                [y_test_original.min(), y_test_original.max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('Actual Turbidity (NTU)', fontsize=12)
axes[0, 0].set_ylabel('Predicted Turbidity (NTU)', fontsize=12)
axes[0, 0].set_title(f'Predicted vs Actual (R² = {r2:.3f})', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Time series plot: First 200 samples
n_samples = min(200, len(y_test_original))
axes[0, 1].plot(range(n_samples), y_test_original[:n_samples], label='Actual', linewidth=2, alpha=0.8)
axes[0, 1].plot(range(n_samples), y_pred[:n_samples], label='Predicted', linewidth=2, alpha=0.8)
axes[0, 1].set_xlabel('Sample Index', fontsize=12)
axes[0, 1].set_ylabel('Turbidity (NTU)', fontsize=12)
axes[0, 1].set_title('Actual vs Predicted Turbidity (Sample)', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Residual plot
residuals = y_test_original - y_pred
axes[1, 0].scatter(y_pred, residuals, alpha=0.5, s=20, c='darkgreen')
axes[1, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Predicted Turbidity (NTU)', fontsize=12)
axes[1, 0].set_ylabel('Residual (Actual - Predicted)', fontsize=12)
axes[1, 0].set_title('Residual Plot', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Residual distribution
axes[1, 1].hist(residuals, bins=50, color='teal', edgecolor='white', alpha=0.8)
axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 1].axvline(x=np.mean(residuals), color='orange', linestyle='-', linewidth=2, label=f'Mean: {np.mean(residuals):.3f}')
axes[1, 1].set_xlabel('Residual (NTU)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Residual Distribution', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/lstm_evaluation_plots.png', dpi=150, bbox_inches='tight')
plt.show()


## 6. Export Predictions for Dashboard


In [ ]:
# Create predictions dataframe for Tableau dashboard
# Get timestamps for test set predictions
test_start_idx = train_size + val_size + SEQUENCE_LENGTH + FORECAST_HORIZON - 1
timestamps = df_lstm['Timestamp'].iloc[test_start_idx:test_start_idx + len(y_test_original)].values

predictions_df = pd.DataFrame({
    'Timestamp': timestamps,
    'Actual_Turbidity': y_test_original,
    'Predicted_Turbidity': y_pred,
    'Prediction_Error': residuals,
    'Absolute_Error': np.abs(residuals)
})

# Add derived columns
predictions_df['Station'] = best_station
predictions_df['Prediction_Date'] = pd.to_datetime(predictions_df['Timestamp']).dt.date

# Calculate daily metrics
daily_metrics = predictions_df.groupby('Prediction_Date').agg({
    'Actual_Turbidity': 'mean',
    'Predicted_Turbidity': 'mean',
    'Absolute_Error': 'mean'
}).reset_index()
daily_metrics.columns = ['Date', 'Avg_Actual_Turbidity', 'Avg_Predicted_Turbidity', 'Avg_MAE']

# Save predictions
predictions_df.to_csv('../outputs/lstm_predictions.csv', index=False)
daily_metrics.to_csv('../outputs/lstm_daily_metrics.csv', index=False)

print("Saved predictions to: ../outputs/lstm_predictions.csv")
print("Saved daily metrics to: ../outputs/lstm_daily_metrics.csv")
print(f"\nPredictions shape: {predictions_df.shape}")
predictions_df.head(10)


## Summary

This notebook implemented an LSTM deep learning model for water turbidity prediction:

**Model Architecture:**
- Built from scratch using TensorFlow/Keras (not pre-trained)
- 2-layer stacked LSTM with dropout regularization
- Input: 48-hour sequences of sensor data (Turbidity, Conductivity, Temperature)
- Output: 24-hour ahead turbidity prediction

**Key Results:**
- The model learns temporal patterns in water quality data
- Predictions can be used for early warning systems
- Results exported for Tableau dashboard visualization

**Next Steps:**
- See Notebook 03 for Water Quality Classification model
- Dashboard data ready for visualization in Tableau Public
